# **About Dataset**

how much you pay for health insurance are not within your control.There are some factors that affect how much health insurance cost.
This dataset contains person's information like age,sex,gender,bmi,region,smoke or not and we have to predict their medical insurance cost.In this notebook I will apply regression techniques of supervised learning to predict the medical insurance costs.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Import Libraries**

In [ ]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from colorama import Fore, Back, Style 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error

from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
import plotly.graph_objs as gobj

init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff

import xgboost
import lightgbm

# **Import Dataset**

In [ ]:
insurance_data = pd.read_csv('/kaggle/input/insurance/insurance.csv')
insurance_data.head()

# **Exploratory Data Analysis**

**Let's check for missing values**

In [ ]:
insurance_data.isnull().sum()

**Data summary**

In [ ]:
# description of the data

insurance_data.describe()

* Age is ranging from 18 to 64 ,with mean of 38.2 and standard deviation of 14.04
* bmi is ranging from 15.96 to 53.13 , with mean of 30.6 and standard deviation of 6.09
* charges is ranging from 1121 to 63770 , with mean of 13270 and standard deviation of 12110

# Charge distribution

In [ ]:
charge_dist =[insurance_data['charges'].values]
group_labels = ['charges'] 

fig = ff.create_distplot(charge_dist, group_labels)
fig.update_layout(title_text='Charge Distribution plot')
fig.show()

Maximum number of Costs/Charges are of 3000-8000

In [ ]:
fig = px.box(insurance_data, x='smoker', y='charges', points="all")
fig.update_layout(title_text="Charge Spread - Smoker and Non Smoker")
fig.show()

* Charges for smoker ranges between 12.82k to 63.77k
* Charges for non smoker ranges between 1.1k to 36k

**Charges Region wise**

In [ ]:
fig = px.box(insurance_data, x='region', y='charges', points="all")
fig.update_layout(title_text="Region wise Charge Spread")
fig.show()

* Charges for Southwest Region ranges between 1.2k to 52.59k
* Charges for Southeast Region ranges between 1.1k to 63.77k
* Charges for Northwest Region ranges between 1.6k to 60.02k
* Charges for Northeast Region ranges between 1.6k to 58.57k

In [ ]:
fig = px.violin(insurance_data, y="charges", x="region", color='sex', box=True, points="all", hover_data=insurance_data.columns)
fig.update_layout(title_text="Analysis in Charges and Region on Sex Status")
fig.show()

* Charges of Southwest Region for female 1.7k to 46k and for male is 1.2 to 52k
* Charges of Southeast Region for female 1.6k to 63k and for male is 1.1 to 62k
* Charges of Northwest Region for female 2.1k to 55k and for male is 1.6 to 60k
* Charges of Northeast Region for female 2.1k to 58k and for male is 1.7 to 48k

In [ ]:
fig = px.violin(insurance_data, y="charges", x="region", color='smoker', box=True, points="all", hover_data=insurance_data.columns)
fig.update_layout(title_text="Analysis in Charges and Region on Smoker Status")
fig.show()

* Charges of Southwest Region for smoker 14k to 53k and for non-smoker is 1.2 to 36k
* Charges of Southeast Region for smoker 16k to 63k and for non-smoker is 1.1 to 36k
* Charges of Northwest Region for smoker 12k to 53k and for non-smoker is 1.6 to 32k
* Charges of Northeast Region for smoker 14k to 60k and for non-smoker is 1.6 to 33k

**charges for childrens region wise**

In [ ]:
fig = px.violin(insurance_data, y="charges", x="region", color='children', box=True, points="all", hover_data=insurance_data.columns)
fig.update_layout(title_text="Analysis in Charges and Region on Children Status")
fig.show()

# Age Distribution

In [ ]:
# age distribution

age_dist =[insurance_data["age"].values]
group_labels = ['age'] 

fig = ff.create_distplot(age_dist, group_labels)
fig.update_layout(title_text='Age Distribution plot')
fig.show()

* As you can see age is normally distrtibuted
* Maximum number of patients are of 18-20 age

In [ ]:
fig = px.box(insurance_data, y='age', points="all")
fig.update_layout(title_text="Age Spread")
fig.show()

Age is ranging from 18 to 64 ,with mean of 39

**Gender wise age distribution**

In [ ]:
fig = px.box(insurance_data, x='sex', y='age', points="all")
fig.update_layout(title_text="Gender wise Age Spread - Male and Female")
fig.show()

Age is ranging from 18 to 64 ,with mean of 40 for female and 39 for male.

In [ ]:
fig = px.violin(insurance_data, y="age", x="sex", color='smoker', box=True, points="all", hover_data=insurance_data.columns)
fig.update_layout(title_text="Analysis in Age and sex on Smoker Status")
fig.show()

In [ ]:
x1 = insurance_data[(insurance_data.sex == 'male')]["age"]
x2 = insurance_data[(insurance_data.sex == 'female')]["age"]

group_labels = ['male', 'female']
colors = ['#F66095', '#A6ACEC']
fig = ff.create_distplot([x1, x2], group_labels, bin_size=.5,
                         curve_type='normal',colors=colors)

fig.update_layout(title_text='Age Distribution with Gender')
fig.show()

**Age distribution of Smoker vs Non-Smoker**

In [ ]:
fig = px.box(insurance_data, x='smoker', y='age', points='all')
fig.update_layout(title_text='Age Spread - Smoker and Non smoker')
fig.show()

Age is ranging from 18 to 64 ,with mean of 38 for smoker and 40 for non-smoker.

In [ ]:
x1 = insurance_data[(insurance_data.smoker == 'yes')]["age"]
x2 = insurance_data[(insurance_data.smoker == 'no')]["age"]

group_labels = ['yes', 'no']
colors = ['#835AF1', '#7FA6EE']
fig = ff.create_distplot([x1, x2], group_labels, bin_size=.5, curve_type='normal',colors=colors)

fig.update_layout(title_text='Age Distribution with Smoker and Non Smoker')
fig.show()

# **Age vs charges**

**Age vs charges for smokers**

In [ ]:
fig = px.scatter(insurance_data[insurance_data.smoker=='yes'], x='age', y='charges')
fig.update_layout(title_text='Analysis in Age and Charges')
fig.show()

In [ ]:
fig = px.density_contour(insurance_data[insurance_data.smoker=='yes'], x='age', y='charges', marginal_x='box', marginal_y='box')
fig.show()

**Age vs charges for smokers**

In [ ]:
fig = px.scatter(insurance_data[insurance_data.smoker=='no'], x='age', y='charges')
fig.update_layout(title_text='Analysis in Age and Charges')
fig.show()

In [ ]:
fig = px.density_contour(insurance_data[insurance_data.smoker=='no'], x='age', y='charges', marginal_x='box', marginal_y='box')
fig.show()

# **BMI Analysis**

**Age and BMI on sex status**

In [ ]:
fig = px.scatter(insurance_data, x='age', y='bmi', color='sex')
fig.update_layout(title_text='Analysis in Age and BMI on Sex Status')
fig.show()

**Charges and BMI on sex status**

In [ ]:
fig = px.scatter(insurance_data, x='bmi', y='charges', color='sex')
fig.update_layout(title_text='Analysis in Charges and BMI on Sex Status')
fig.show()

In [ ]:
fig = px.density_contour(insurance_data, x='bmi', y='charges', marginal_x='box', marginal_y='box', color='sex')
fig.show()

# **Correlation matrix**

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(insurance_data.corr(), vmin=-1, cmap='coolwarm', annot=True);

In [ ]:
# Converting category labels into numerical using LabelEncoder

#sex
le = LabelEncoder()
le.fit(insurance_data.sex.drop_duplicates()) 
insurance_data.sex = le.transform(insurance_data.sex)
# smoker or not
le.fit(insurance_data.smoker.drop_duplicates()) 
insurance_data.smoker = le.transform(insurance_data.smoker)
#region
le.fit(insurance_data.region.drop_duplicates()) 
insurance_data.region = le.transform(insurance_data.region)

# **Data Modelling**

In [ ]:
insurance_data.head()

**Train Test split**

In [ ]:
X = insurance_data.drop(['charges','region', 'sex', 'children'], axis = 1)
y = insurance_data.charges

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

**Linear Regression**

In [ ]:
linear_reg = LinearRegression()
linear_reg.fit(X_train,y_train)
y_test_pred = linear_reg.predict(X_test)

In [ ]:
print(Fore.GREEN + "Accuracy of Linear Regression is : ",linear_reg.score(X_test,y_test))

**Random Forest Regressor**

In [ ]:
random_reg = RandomForestRegressor(random_state = 2)
random_reg.fit(X_train,y_train)
random_test_pred = random_reg.predict(X_test)

In [ ]:
print(Fore.GREEN + "Accuracy of Random Forest Regression is : ",random_reg.score(X_test,y_test))

**Xboost Regressor**

In [ ]:
xgb_reg = xgboost.XGBRegressor(max_depth=2, random_state=1)
xgb_reg.fit(X_train,y_train)
xgb_test = xgb_reg.predict(X_test)

In [ ]:
print(Fore.GREEN + "Accuracy of XGB Regression is : ",xgb_reg.score(X_test,y_test))

**Lightgbm**

In [ ]:
lgb_reg = lightgbm.LGBMRegressor(max_depth=2, random_state=1)
lgb_reg.fit(X_train,y_train)
lgb_test = lgb_reg.predict(X_test)

In [ ]:
print(Fore.GREEN + "Accuracy of LGB Regression is : ",lgb_reg.score(X_test,y_test))

In [ ]:
from catboost import CatBoostRegressor
catb_reg = CatBoostRegressor()
catb_reg.fit(X_train,y_train)
catb_test = catb_reg.predict(X_test)

In [ ]:
print(Fore.GREEN + "Accuracy of CatBoost Regression is : ",catb_reg.score(X_test,y_test))

* LGBM Regressor : 89.79%
* CatBoost Regressor : 88.76%
* XGB Regressor: 88.65%
* Random Forest Regressor: 86.06%
* Linear Regressor: 79.45%

# If you found this kernel helpful, please upvote it